In [2]:
import requests
import os
import gzip
from tqdm.notebook import trange
import subprocess


#Importer les données IMDB
# liste des urls pour chaque fichier à telecharger
# urls = ['https://datasets.imdbws.com/name.basics.tsv.gz',
#        'https://datasets.imdbws.com/title.basics.tsv.gz', 
#        'https://datasets.imdbws.com/title.episode.tsv.gz', 
#        'https://datasets.imdbws.com/title.principals.tsv.gz', 
#        'https://datasets.imdbws.com/title.ratings.tsv.gz']

# # telechargement des fichiers
# for url in urls:
#     filename = url.split('/')[-1]
#     target_path = os.path.join("..", "data", "raw", filename)

#     response = requests.get(url, stream=True)  
   
#     if response.status_code == 200:
#         with open(target_path, 'wb') as f:
#             f.write(response.raw.read())
#     tsv_path = target_path.replace(".gz", "")

#     with gzip.open(target_path, 'rb') as gz_file:
#         with open(tsv_path, 'wb') as f:
#             f.write(gz_file.read())
            
#     for i in trange(1, desc='Statut'):
#         print('Fichier téléchargé :', tsv_path)
   

In [3]:
# Demarrer le conteneur Docker pour MySQL
print("🚀 Starting Docker container...")
COMPOSE_FILE="../docker/docker-compose.yml"
subprocess.run(["docker", "compose", "-f", COMPOSE_FILE, "up", "-d", "--build"])

🚀 Starting Docker container...


 Image docker-mysql Building 


#1 [internal] load local bake definitions
#1 reading from stdin 558B done
#1 DONE 0.0s

#2 [internal] load build definition from Dockerfile.mysql
#2 transferring dockerfile: 285B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/mysql:8.0
#3 ...

#4 [auth] library/mysql:pull token for registry-1.docker.io
#4 DONE 0.0s

#3 [internal] load metadata for docker.io/library/mysql:8.0
#3 DONE 1.0s

#5 [internal] load .dockerignore
#5 transferring context: 2B done
#5 DONE 0.0s

#6 [1/1] FROM docker.io/library/mysql:8.0@sha256:99d774bf02a48a1bb1c599920d2571946d31e5940b854b02737d5e95c184358f
#6 resolve docker.io/library/mysql:8.0@sha256:99d774bf02a48a1bb1c599920d2571946d31e5940b854b02737d5e95c184358f done
#6 CACHED

#7 exporting to image
#7 exporting layers done
#7 exporting manifest sha256:cc0547b9a74fe963a4486a74cf94975e891023bb7340e940939d9030dc7715c7 done
#7 exporting config sha256:c5bfc23de452870463110aea77e9e84417f5e2bb10e662033d28b4d55c2c6b54 done
#7 exporting attestati

 Image docker-mysql Built 
 Network docker_default Creating 
 Network docker_default Created 
 Container moviesdb_mysql Creating 
 Container moviesdb_mysql Created 
 Container moviesdb_mysql Starting 
 Container moviesdb_mysql Started 


CompletedProcess(args=['docker', 'compose', '-f', '../docker/docker-compose.yml', 'up', '-d', '--build'], returncode=0)

In [4]:
#Copier les fichiers SQL dans le conteneur
CONTAINER = "moviesdb_mysql"
subprocess.run(["docker", "cp", "../data/raw/", f"{CONTAINER}:/tmp/raw/"])
subprocess.run(["docker", "cp" , "../mysql/imdb-create-db.sql", f"{CONTAINER}:/tmp/"])
subprocess.run(["docker", "cp", "../mysql/imdb-create-tables.sql", f"{CONTAINER}:/tmp/"])
subprocess.run(["docker", "cp", "../mysql/imdb-load-data.sql", f"{CONTAINER}:/tmp/"])
subprocess.run(["docker", "cp", "../mysql/imdb-add-constraints.sql", f"{CONTAINER}:/tmp/"])
subprocess.run(["docker", "cp", "../mysql/imdb-add-index.sql", f"{CONTAINER}:/tmp/"])


CompletedProcess(args=['docker', 'cp', '../mysql/imdb-add-index.sql', 'moviesdb_mysql:/tmp/'], returncode=0)

In [5]:
subprocess.run([
    "docker", "exec", "-i", CONTAINER,
    "mysql", "-u", "root", "-proot",
    "-e", "SOURCE /tmp/imdb-create-db.sql"
], check=True)

mysql: [Warning] Using a password on the command line interface can be insecure.


CompletedProcess(args=['docker', 'exec', '-i', 'moviesdb_mysql', 'mysql', '-u', 'root', '-proot', '-e', 'SOURCE /tmp/imdb-create-db.sql'], returncode=0)

In [6]:
subprocess.run([
    "docker", "exec", "-i", CONTAINER,
    "mysql", "-u", "root", "-proot",
    "IMDb",
    "-e", "SOURCE /tmp/imdb-create-tables.sql"
], check=True)

mysql: [Warning] Using a password on the command line interface can be insecure.


CompletedProcess(args=['docker', 'exec', '-i', 'moviesdb_mysql', 'mysql', '-u', 'root', '-proot', 'IMDb', '-e', 'SOURCE /tmp/imdb-create-tables.sql'], returncode=0)

In [7]:
subprocess.run([
    "docker", "exec", "-i", CONTAINER,
    "mysql", "-u", "root", "-proot",
    "--local-infile=1",
    "IMDb",
    "-e", "SOURCE /tmp/imdb-load-data.sql"
], check=True)

mysql: [Warning] Using a password on the command line interface can be insecure.


CompletedProcess(args=['docker', 'exec', '-i', 'moviesdb_mysql', 'mysql', '-u', 'root', '-proot', '--local-infile=1', 'IMDb', '-e', 'SOURCE /tmp/imdb-load-data.sql'], returncode=0)

In [ ]:
subprocess.run([
    "docker", "exec", "-i", CONTAINER,
    "mysql", "-u", "root", "-proot",
    "IMDb",
    "-e", "SOURCE /tmp/imdb-add-constraints.sql"
], check=True)

In [ ]:
subprocess.run([
    "docker", "exec", "-i", CONTAINER,
    "mysql", "-u", "root", "-proot",
    "IMDb",
    "-e", "SOURCE /tmp/imdb-add-index.sql"
], check=True)

In [ ]:
# #Arreter le conteneur Docker

print("🚀 Stopping Docker container")
COMPOSE_FILE="../docker/docker-compose.yml"
CONTAINER_NAME="moviesdb_mysql"

subprocess.run(["docker", "stop", CONTAINER_NAME])
subprocess.run(["docker", "compose", "-f", COMPOSE_FILE, "down"])